In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("assignment").getOrCreate()

In [0]:
spark = SparkSession.builder.appName('bank').getOrCreate()


In [0]:
from pyspark.sql.functions import *

In [0]:
df_customer = spark.read.csv("dbfs:/FileStore/shared_uploads/mafrozmohiuddin@deloitte.com/bank_customer-1.csv",header=True,inferSchema=True)

In [0]:
df_customer.display()

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
df_fixed_deposit = spark.read.csv("dbfs:/FileStore/shared_uploads/mafrozmohiuddin@deloitte.com/bank_fd_account-1.csv",header=True,inferSchema=True)

In [0]:
df_fixed_deposit.display()

ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS
5212340001,1002,22-JUN-00,30,15000,Closed
5212340002,1002,11-MAY-02,66,25000,Active
5212340003,1007,26-NOV-03,36,80555,Closed
5212340004,1003,29-NOV-04,60,30000,Active
5212340005,1009,23-JUN-04,120,45000,Active
5212340006,1001,26-SEP-05,24,45000,Active
5212340007,1010,18-OCT-05,84,15000000,Active
5212340008,1008,18-DEC-06,36,98765,Active
5212340009,1006,14-FEB-07,120,25000,Active
5212340010,1007,21-SEP-08,60,2500000,Active


In [0]:
df_recurrent_deposit = spark.read.csv("dbfs:/FileStore/shared_uploads/mafrozmohiuddin@deloitte.com/bank_rd_account.csv",header=True,inferSchema=True)

In [0]:
df_recurrent_deposit.display()

ACCOUNT_NO,RD_CUST_ID,RD_START_DT,RD_TERM_YRS,INSTALLMENT_AMT,CURR_BALANCE,ACC_STATUS
6000010001,1006,13-JUN-06,10,1000,75000,Active
6000010002,1003,26-NOV-06,6,3000,208000,Active
6000010003,1008,03-OCT-07,5,2500,148000,Active
6000010004,1009,29-APR-07,4,6000,0,Closed
6000010005,1001,17-FEB-08,1,10000,0,Closed
6000010006,1007,16-JUL-08,9,20000,1000012,Active
6000010007,1001,26-MAY-08,4,5000,0,Closed
6000010008,1010,18-DEC-09,7,50000,1550000,Active
6000010009,1007,20-APR-09,4,14000,560000,Active
6000010010,1010,08-MAY-10,7,25000,700000,Active


In [0]:
df_saving_deposit= spark.read.csv("dbfs:/FileStore/shared_uploads/mafrozmohiuddin@deloitte.com/bank_sb_account.csv",header=True,inferSchema=True)

In [0]:
df_saving_deposit.display()

ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE
1000012003,1005,1001,0.0,Closed,20-NOV-06,23-AUG-11
1000012004,1008,null,84924.0,Active,11-MAR-06,null
1000012005,1009,1004,209844.03,Active,20-NOV-06,null
1000012006,1004,null,0.0,Closed,12-SEP-07,12-SEP-10
1000012007,1003,null,12300.0,Inactive,09-DEC-08,null
1000012008,1005,null,10235.0,Active,23-MAY-09,null
1000012009,1006,null,73535.0,Active,10-OCT-10,null
1000012010,1004,null,54674.0,Active,09-OCT-11,null
1000012001,1001,null,25000.0,Active,06-SEP-04,null
1000012002,1002,null,31109.43,Active,16-DEC-05,null


In [0]:
df_customer.columns

Out[61]: ['CUST_ID',
 'CUST_FNAME',
 'INITIALS',
 'CUST_LNAME',
 'CUST_SEX',
 'CUST_DOB',
 'CUST_TYPE']

In [0]:
df_customer.createOrReplaceTempView('customer')
df_fixed_deposit.createOrReplaceTempView('fixed_deposit')
df_saving_deposit.createOrReplaceTempView('saving_deposit')
df_recurrent_deposit.createOrReplaceTempView('recurrent_deposit')

In [0]:
# A -----------------------------------------
%sql
select cust_id, cust_fname, account_no, initial_amt from customer
natural join fixed_deposit
order by initial_amt desc




cust_id,cust_fname,account_no,initial_amt
1010,Popular Enterprises,5212340007,15000000
1007,DealWell Co.,5212340010,2500000
1008,Aman,5212340008,98765
1007,DealWell Co.,5212340003,80555
1009,Smitha,5212340005,45000
1001,John,5212340006,45000
1003,Alice,5212340004,30000
1002,Franklin,5212340002,25000
1006,Ahmad,5212340009,25000
1002,Franklin,5212340001,15000


In [0]:
df_customer.join(df_fixed_deposit,'cust_id','inner').\
filter((col('initial_amt')>25000) & (col('acc_status')=='Active')).\
select('cust_id','cust_fname','account_no','initial_amt').\
orderBy(col('initial_amt').desc()).\
show()

+-------+-------------------+----------+-----------+
|cust_id|         cust_fname|account_no|initial_amt|
+-------+-------------------+----------+-----------+
|   1010|Popular Enterprises|5212340007|   15000000|
|   1007|       DealWell Co.|5212340010|    2500000|
|   1008|               Aman|5212340008|      98765|
|   1009|             Smitha|5212340005|      45000|
|   1001|               John|5212340006|      45000|
|   1003|              Alice|5212340004|      30000|
+-------+-------------------+----------+-----------+



In [0]:
# B ------------------------------
%sql
select c.cust_id as CustomerID,c.cust_fname as CustomerFirstName,f.account_no as AccountNo,
       f.initial_amt as FD_InitialBal,r.curr_balance as RD_CurrentBal,s.curr_bal_amt as SB_CurrentBal,
       (f.initial_amt + r.curr_balance + s.curr_bal_amt) as NetWorth --getting required cols only

from customer c inner join fixed_deposit f on c.cust_id=f.cust_id 

inner join saving_deposit s on s.primary_cust_id=f.cust_id 

inner join recurrent_deposit r on s.primary_cust_id=r.rd_cust_id --joining 3 tables on customer ID

where ((f.acc_status=='Active') and (s.acc_status=='Active') and (r.acc_status=='Active')) --filtering based on Active accounts

order by NetWorth desc; --ordering


CustomerID,CustomerFirstName,AccountNo,FD_InitialBal,RD_CurrentBal,SB_CurrentBal,NetWorth
1008,Aman,5212340008,98765,148000,84924.0,331689.0
1006,Ahmad,5212340009,25000,75000,73535.0,173535.0


In [0]:
%sql
--no op when using secondary to join
select c.cust_id,c.cust_fname,f.account_no,f.initial_amt as FD_InitialBal,r.curr_balance as RD_CurrentBal,s.curr_bal_amt as SB_CurrentBal,
(f.initial_amt + r.curr_balance + s.curr_bal_amt) as NetWorth
from customer c inner join fixed_deposit f on c.cust_id=f.cust_id 
inner join saving_deposit s on s.secondary_cust_id=f.cust_id 
inner join recurrent_deposit r on s.secondary_cust_id=r.rd_cust_id
where ((f.acc_status=='Active') and (s.acc_status=='Active') and (r.acc_status=='Active'))
order by NetWorth desc;
     

cust_id,cust_fname,account_no,FD_InitialBal,RD_CurrentBal,SB_CurrentBal,NetWorth


In [0]:
df_customer.join(df_fixed_deposit,df_customer['cust_id'] == df_fixed_deposit['cust_id'],'inner').\
join(df_saving_deposit,df_customer['cust_id'] == df_saving_deposit['primary_cust_id'],'inner').\
join(df_recurrent_deposit,df_saving_deposit['primary_cust_id'] == df_recurrent_deposit['rd_cust_id'],'inner').\
filter((df_fixed_deposit['acc_status']=='Active') & (df_saving_deposit['acc_status']=='Active') &(df_recurrent_deposit['acc_status']=='Active')).\
withColumn('Net Worth',(df_fixed_deposit['initial_amt'] + df_recurrent_deposit['curr_balance'] + df_saving_deposit['curr_bal_amt'])).\
select(df_customer['cust_id'].alias('CustomerID'),df_customer['cust_fname'].alias('CustomerFirstName'),df_fixed_deposit['account_no'],df_fixed_deposit['initial_amt'] .alias("FD_InitialBal"),df_recurrent_deposit['curr_balance'].alias('RD_CurrentBal'),df_saving_deposit['curr_bal_amt'].alias('SB_CurrentBal'),'Net Worth').\
orderBy(col('Net Worth').desc()).\
show()
     

+----------+-----------------+----------+-------------+-------------+-------------+---------+
|CustomerID|CustomerFirstName|account_no|FD_InitialBal|RD_CurrentBal|SB_CurrentBal|Net Worth|
+----------+-----------------+----------+-------------+-------------+-------------+---------+
|      1008|             Aman|5212340008|        98765|       148000|      84924.0| 331689.0|
|      1006|            Ahmad|5212340009|        25000|        75000|      73535.0| 173535.0|
+----------+-----------------+----------+-------------+-------------+-------------+---------+



In [0]:
# C ------------------------------------------
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
     
     

In [0]:
result = df_customer.select('CUST_TYPE')

In [0]:
from pyspark.sql.functions import when
result_final = result.withColumn("CUST_TYPE", when(df_customer.CUST_TYPE == "IND", 1).when(df_customer.CUST_TYPE == "CORP", 2)).withColumnRenamed("CUST_TYPE","CUST_TYPE_INDEX").show()

+---------------+
|CUST_TYPE_INDEX|
+---------------+
|              1|
|              1|
|              1|
|              1|
|              1|
|              1|
|              2|
|              1|
|              1|
|              2|
+---------------+



In [0]:
# E -----------------------------------
%sql
--overview
select *
from customer c 
left join fixed_deposit f on c.cust_id=f.cust_id
left join saving_deposit s on c.cust_id=s.primary_cust_id
left join recurrent_deposit r on c.cust_id=r.rd_cust_id
where( f.acc_status=='Active' and s.acc_status=='Active' and r.acc_status=='Active') or ( f.acc_status=='Active' or s.acc_status=='Active')
order by c.cust_id,f.account_no,s.account_no,r.account_no;
     

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE,ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS,ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE,ACCOUNT_NO,RD_CUST_ID,RD_START_DT,RD_TERM_YRS,INSTALLMENT_AMT,CURR_BALANCE,ACC_STATUS
1001,John,null,Smith,M,21-OCT-78,IND,5212340006,1001,26-SEP-05,24,45000,Active,1000012001,1001,null,25000.0,Active,06-SEP-04,null,6000010005,1001,17-FEB-08,1,10000,0,Closed
1001,John,null,Smith,M,21-OCT-78,IND,5212340006,1001,26-SEP-05,24,45000,Active,1000012001,1001,null,25000.0,Active,06-SEP-04,null,6000010007,1001,26-MAY-08,4,5000,0,Closed
1002,Franklin,J,Dang,M,21-OCT-78,IND,5212340001,1002,22-JUN-00,30,15000,Closed,1000012002,1002,null,31109.43,Active,16-DEC-05,null,null,null,null,null,null,null,null
1002,Franklin,J,Dang,M,21-OCT-78,IND,5212340002,1002,11-MAY-02,66,25000,Active,1000012002,1002,null,31109.43,Active,16-DEC-05,null,null,null,null,null,null,null,null
1003,Alice,M,Powell,F,09-APR-76,IND,5212340004,1003,29-NOV-04,60,30000,Active,1000012007,1003,null,12300.0,Inactive,09-DEC-08,null,6000010002,1003,26-NOV-06,6,3000,208000,Active
1004,Ramesh,R,Narayan,M,08-DEC-71,IND,null,null,null,null,null,null,1000012010,1004,null,54674.0,Active,09-OCT-11,null,null,null,null,null,null,null,null
1005,Nancy,S,Taylor,F,17-AUG-79,IND,null,null,null,null,null,null,1000012008,1005,null,10235.0,Active,23-MAY-09,null,null,null,null,null,null,null,null
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND,5212340009,1006,14-FEB-07,120,25000,Active,1000012009,1006,null,73535.0,Active,10-OCT-10,null,6000010001,1006,13-JUN-06,10,1000,75000,Active
1007,DealWell Co.,null,null,null,null,CORP,5212340010,1007,21-SEP-08,60,2500000,Active,null,null,null,null,null,null,null,6000010006,1007,16-JUL-08,9,20000,1000012,Active
1007,DealWell Co.,null,null,null,null,CORP,5212340010,1007,21-SEP-08,60,2500000,Active,null,null,null,null,null,null,null,6000010009,1007,20-APR-09,4,14000,560000,Active


In [0]:
%sql
--the reason wecannot use inner join is it would return cust who has accnt in all 3 only
select *
from customer c 
inner join fixed_deposit f on c.cust_id=f.cust_id
inner join saving_deposit s on c.cust_id=s.primary_cust_id
inner join recurrent_deposit r on c.cust_id=r.rd_cust_id
order by c.cust_id,f.account_no,s.account_no,r.account_no;

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE,ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS,ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE,ACCOUNT_NO,RD_CUST_ID,RD_START_DT,RD_TERM_YRS,INSTALLMENT_AMT,CURR_BALANCE,ACC_STATUS
1001,John,null,Smith,M,21-OCT-78,IND,5212340006,1001,26-SEP-05,24,45000,Active,1000012001,1001,null,25000.0,Active,06-SEP-04,null,6000010005,1001,17-FEB-08,1,10000,0,Closed
1001,John,null,Smith,M,21-OCT-78,IND,5212340006,1001,26-SEP-05,24,45000,Active,1000012001,1001,null,25000.0,Active,06-SEP-04,null,6000010007,1001,26-MAY-08,4,5000,0,Closed
1003,Alice,M,Powell,F,09-APR-76,IND,5212340004,1003,29-NOV-04,60,30000,Active,1000012007,1003,null,12300.0,Inactive,09-DEC-08,null,6000010002,1003,26-NOV-06,6,3000,208000,Active
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND,5212340009,1006,14-FEB-07,120,25000,Active,1000012009,1006,null,73535.0,Active,10-OCT-10,null,6000010001,1006,13-JUN-06,10,1000,75000,Active
1008,Aman,K,Mathur,M,30-OCT-80,IND,5212340008,1008,18-DEC-06,36,98765,Active,1000012004,1008,null,84924.0,Active,11-MAR-06,null,6000010003,1008,03-OCT-07,5,2500,148000,Active
1009,Smitha,null,Ramesh,F,03-MAR-76,IND,5212340005,1009,23-JUN-04,120,45000,Active,1000012005,1009,1004,209844.03,Active,20-NOV-06,null,6000010004,1009,29-APR-07,4,6000,0,Closed


In [0]:
%sql
(select c.cust_id as CustomerID,f.account_no as AccountNo,f.initial_amt as BalanceAmmount,f.acc_status as AccountStatus
from customer c 
left join fixed_deposit f on c.cust_id=f.cust_id
where f.acc_status=='Active')  --left join customer with fd and store 4 needed cols

                                union
                                
(select c.cust_id,s.account_no,s.curr_bal_amt,s.acc_status
from customer c 
left join saving_deposit s on c.cust_id=s.primary_cust_id
where s.acc_status=='Active')  --left join customer with SB and store 4 needed cols
                               union
                               
(select c.cust_id,r.account_no,r.curr_balance,r.acc_status
from customer c 
left join recurrent_deposit r on c.cust_id=r.rd_cust_id
where r.acc_status=='Active')  --left join customer with RD and store 4 needed cols

--union all 3 outputs

order by CustomerID,AccountNo; --order as needed
     

CustomerID,AccountNo,BalanceAmmount,AccountStatus
1001,1000012001,25000.0,Active
1001,5212340006,45000.0,Active
1002,1000012002,31109.43,Active
1002,5212340002,25000.0,Active
1003,5212340004,30000.0,Active
1003,6000010002,208000.0,Active
1004,1000012010,54674.0,Active
1005,1000012008,10235.0,Active
1006,1000012009,73535.0,Active
1006,5212340009,25000.0,Active


In [0]:
df1=df_customer.join(df_fixed_deposit,df_customer['cust_id'] == df_fixed_deposit['cust_id'],'left').filter(col('acc_status')=='Active')
df2=df_customer.join(df_saving_deposit,df_customer['cust_id'] == df_saving_deposit['primary_cust_id'],'left').filter(col('acc_status')=='Active')
df3=df_customer.join(df_recurrent_deposit,df_customer['cust_id'] == df_recurrent_deposit['rd_cust_id'],'left').filter(col('acc_status')=='Active')

df1=df1.select(df_customer['cust_id'],'account_no','initial_amt','acc_status')
df2=df2.select(df_customer['cust_id'],'account_no','curr_bal_amt','acc_status')
df3=df3.select(df_customer['cust_id'],'account_no','curr_balance','acc_status')
     

In [0]:
final_df=df1.union(df2).union(df3).orderBy('cust_id','account_no')
final_df.count()

Out[93]: 22

In [0]:
final_df.show()

+-------+----------+-----------+----------+
|cust_id|account_no|initial_amt|acc_status|
+-------+----------+-----------+----------+
|   1001|1000012001|    25000.0|    Active|
|   1001|5212340006|    45000.0|    Active|
|   1002|1000012002|   31109.43|    Active|
|   1002|5212340002|    25000.0|    Active|
|   1003|5212340004|    30000.0|    Active|
|   1003|6000010002|   208000.0|    Active|
|   1004|1000012010|    54674.0|    Active|
|   1005|1000012008|    10235.0|    Active|
|   1006|1000012009|    73535.0|    Active|
|   1006|5212340009|    25000.0|    Active|
|   1006|6000010001|    75000.0|    Active|
|   1007|5212340010|  2500000.0|    Active|
|   1007|6000010006|  1000012.0|    Active|
|   1007|6000010009|   560000.0|    Active|
|   1008|1000012004|    84924.0|    Active|
|   1008|5212340008|    98765.0|    Active|
|   1008|6000010003|   148000.0|    Active|
|   1009|1000012005|  209844.03|    Active|
|   1009|5212340005|    45000.0|    Active|
|   1010|5212340007|      1.5E7|

In [0]:
# F ---------------------------------------------
df1=df_customer.join(df_fixed_deposit,df_customer['cust_id'] == df_fixed_deposit['cust_id'],'left').filter(col('acc_status')=='Active')
df2=df_customer.join(df_saving_deposit,df_customer['cust_id'] == df_saving_deposit['primary_cust_id'],'left').filter(col('acc_status')=='Active')
df3=df_customer.join(df_recurrent_deposit,df_customer['cust_id'] == df_recurrent_deposit['rd_cust_id'],'left').filter(col('acc_status')=='Active')

df1=df1.select(df_customer['cust_id'],'cust_fname','account_no','initial_amt','acc_status',lit('FD').alias('acc_type'))
df2=df2.select(df_customer['cust_id'],'cust_fname','account_no','curr_bal_amt','acc_status',lit('SB'))
df3=df3.select(df_customer['cust_id'],'cust_fname','account_no','curr_balance','acc_status',lit('RD'))
     

In [0]:
final_df=df1.union(df2).union(df3).orderBy('cust_id','account_no')
final_df.count()
     

Out[111]: 22

In [0]:
final_df.display()


cust_id,cust_fname,account_no,initial_amt,acc_status,acc_type
1001,John,1000012001,25000.0,Active,SB
1001,John,5212340006,45000.0,Active,FD
1002,Franklin,1000012002,31109.43,Active,SB
1002,Franklin,5212340002,25000.0,Active,FD
1003,Alice,5212340004,30000.0,Active,FD
1003,Alice,6000010002,208000.0,Active,RD
1004,Ramesh,1000012010,54674.0,Active,SB
1005,Nancy,1000012008,10235.0,Active,SB
1006,Ahmad,1000012009,73535.0,Active,SB
1006,Ahmad,5212340009,25000.0,Active,FD
